In [2]:
#import BALTIC: repo needs to be cloned in if you don't have it yet.
import imp
bt = imp.load_source('baltic', '../baltic/baltic/baltic.py')

In [4]:
#import necessary libraries
import re
import copy
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
%matplotlib inline

In [12]:
input_tree_path = "../beast/phylogeography/dec18-test-mcc-medianHeights.tree"
posterior_trees_path = "../beast/phylogeography/dec18testzika-usvi-geo-ancestral-state-reconstruction.trees"

In [66]:
#remember to remove burnin from the file if you're using a different file from this one :)
posterior_trees = {}

for line in open(posterior_trees_path,"r"):
    if line.startswith("tree STATE"):
        state = line.split(" ")[1]
        treeString_start=line.index('(') #start of the Newick string defining the tree
        tree = bt.make_tree(line[treeString_start:])
        tree.traverse_tree() #need to traverse tree so that BALTIC can make height/x/y annotations
        tree.sortBranches()
        posterior_trees[state] = tree

print("Found {} trees in the posterior distribution of trees. Remove burnin if you have not already.".format(len(posterior_trees)))

Found 32 trees in the posterior distribution of trees. Remove burnin if you have not already.


In [83]:
#make a list of all of the possible countries that could be the ancestral state at a node
countries = []
for line in open("../config/colors.tsv"):
    if line.startswith("country"):
        countries.append(line.split('\t')[1])

In [113]:
for key in posterior_trees.keys():
    print(posterior_trees[key].root.childHeight)

4.70780493608633
4.782290569245708
4.768097198998374
4.867041989864751
4.625864246873712
4.964254106789892
4.654317959784396
4.815054776541864
4.704573050439786
4.630257011089955
4.629966919523653
4.804453204995197
4.625473297962884
4.698260369208053
4.877567099778642
5.450913770330191
4.69365771867641
4.632502753945949
4.8836094498406055
4.618812471547672
4.482227993656273
4.602029057572953
4.766185572892794
4.7792984214022125
4.56389268481195
4.61889513200958
5.209575898452051
4.963564647911863
4.79806779777128
4.917203816180425
4.632609254336664
4.664028669973466


In [116]:
usvi_introductions_per_posterior_tree = {key:[] for key in posterior_trees.keys()} #dict where subtrees can be grouped by deme they circulate in

#I only want the portion of the subtree that circulated within a deme. 
#Put more simply, I want to see the introduction into a deme, and then only the
#ciruclation within that deme post introduction. 
traverse_if_country_remains_same=lambda w:w.traits["country"]==k_location 

for key in posterior_trees.keys():
    root_year = (2018.0492813141684 - posterior_trees[key].root.childHeight) #root.childHeight is the number of years separating the root and the most reccently sampled tip.
    for k in posterior_trees[key].Objects:
        if k.height == 0:#Exlude the root of the tree, as it has no parent.
            continue
        else:
            k_location = k.traits["country"]
            k_parent_location = k.parent.traits["country"]
            k_decimalDate = root_year + k.height
            k_parent_decimalDate = root_year + k.parent.height
            #print(k_parent_decimalDate)
            if k_location != k_parent_location and k_location == "usvi": #introduction to USVI
                subtree = tree.subtree(k, traverse_condition=traverse_if_country_remains_same)
                usvi_introductions_per_posterior_tree[key].append([k_parent_decimalDate,k_decimalDate, subtree])
    


In [154]:
subtree_export_import_date_data = [] #this will be a list of lists that I'll turn into a df.

for key in usvi_introductions_per_posterior_tree.keys(): #for every tree in the posterior set
    for i in range(len(usvi_introductions_per_posterior_tree[key])):
        subtree = usvi_introductions_per_posterior_tree[key][i][2]
        if len(subtree.Objects) > 1: #clade has more than one child, which means it can be 
            clade_id = "clade_{}".format(len(subtree.Objects))
        elif len(subtree.Objects) == 1:
            for k in subtree.Objects:
                if k.branchType == "leaf":
                    clade_id = "clade_{}".format(k.numName)
        data_list = [clade_id,usvi_introductions_per_posterior_tree[key][i][0],usvi_introductions_per_posterior_tree[key][i][1]]
        subtree_export_import_date_data.append(data_list)
            

clade_392
clade_411
clade_13
clade_398
clade_25
clade_9
clade_420
clade_407
clade_407
clade_420
clade_9
clade_13
clade_398
clade_25
clade_411
clade_2
clade_411
clade_13
clade_398
clade_25
clade_392
clade_9
clade_420
clade_2
clade_392
clade_25
clade_398
clade_13
clade_411
clade_9
clade_420
clade_407
clade_411
clade_13
clade_398
clade_25
clade_392
clade_9
clade_420
clade_407
clade_407
clade_9
clade_420
clade_13
clade_398
clade_25
clade_411
clade_392
clade_407
clade_10
clade_420
clade_392
clade_25
clade_13
clade_398
clade_411
clade_420
clade_9
clade_407
clade_13
clade_398
clade_25
clade_411
clade_392
clade_392
clade_25
clade_13
clade_398
clade_411
clade_2
clade_9
clade_420
clade_9
clade_420
clade_392
clade_398
clade_13
clade_25
clade_411
clade_407
clade_420
clade_10
clade_407
clade_392
clade_411
clade_25
clade_398
clade_13
clade_392
clade_3
clade_25
clade_13
clade_398
clade_2
clade_420
clade_9
clade_2
clade_420
clade_9
clade_392
clade_25
clade_398
clade_13
clade_411
clade_2
clade_9
clade_

In [159]:
data = pd.DataFrame(subtree_export_import_date_data)
data

,0,1,2
0,clade_392,2015.615017,2016.224504
1,clade_411,2015.425468,2016.788501
2,clade_13,2015.786106,2016.233237
3,clade_398,2015.696818,2016.531143
4,clade_25,2015.503324,2015.727142
...,...,...,...
251,clade_398,2015.756559,2016.531143
252,clade_392,2015.734329,2016.224504
253,clade_10,2015.559968,2015.777010
254,clade_420,2015.748908,2016.824093
